In [58]:
#Version #1

# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression

# # Load and preprocess data
# def load_and_preprocess(file_path):
#     data = pd.read_csv(file_path)

#     # Handle missing values (basic example: fill with mean)
#     data.fillna(data.mean(numeric_only=True), inplace=True)

#     # Filter for the most recent season
#     if 'season' in data.columns:
#         most_recent_season = data['season'].max()
#         data = data[data['season'] == most_recent_season]
#     else:
#         raise ValueError("The dataset does not contain a 'season' column.")

    
#     features = [
#         'age', 'experience', 'fg_per_game', 'fga_per_game', 'x3p_per_game',
#         'x2p_per_game', 'ft_per_game', 'trb_per_game', 'ast_per_game', 'pts_per_game'
#     ]

#     return data, features

# # Normalize features
# def normalize_features(X):
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
#     return X_scaled, scaler

# # Predict player performance
# def predict_player_performance(data, model, scaler, player_name, target):
#     player_data = data[data['player'] == player_name]
#     if player_data.empty:
#         return f"Player {player_name} not found in the dataset."

#     features = [
#         'age', 'experience', 'fg_per_game', 'fga_per_game', 'x3p_per_game',
#         'x2p_per_game', 'ft_per_game', 'trb_per_game', 'ast_per_game', 'pts_per_game'
#     ]

#     player_features = player_data[features].values
#     player_features_scaled = scaler.transform(player_features)
#     predicted_value = model.predict(player_features_scaled)

#     return f"Predicted {target} for {player_name}: {predicted_value[0]:.2f}"

# # Train the model for a specific target
# def train_model(data, features, target):
#     X = data[features]
#     y = data[target]

#     # Normalize features
#     X_scaled, scaler = normalize_features(X)

#     # Train the model
#     model = LinearRegression()
#     model.fit(X_scaled, y)

#     return model, scaler

# # Main workflow
# def main():
#     # File path to dataset
#     file_path = '/Users/davislaroque/Desktop/NBA Project/archive/Player Per Game.csv'
#     # Load and preprocess data
#     data, features = load_and_preprocess(file_path)

    
#     # CHANGE TARGET VARIABLE
#     target = 'trb_per_game'
#      # CHANGE PLAYER NAME
#     player_name = "Zach Edey" 

    
#     # Train the model for the specified target
#     model, scaler = train_model(data, features, target)

#     result = predict_player_performance(data, model, scaler, player_name, target)
#     print(result)

# if __name__ == '__main__':
#     main()


In [16]:
# #Version #2

# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import r2_score  # Import r2_score from sklearn.metrics


# # Mapping from team abbreviations to full team names
# team_abbreviation_to_name = {
#     'ATL': 'Atlanta Hawks',
#     'BOS': 'Boston Celtics',
#     'BRK': 'Brooklyn Nets',
#     'CHO': 'Charlotte Hornets',
#     'CHI': 'Chicago Bulls',
#     'CLE': 'Cleveland Cavaliers',
#     'DAL': 'Dallas Mavericks',
#     'DEN': 'Denver Nuggets',
#     'DET': 'Detroit Pistons',
#     'GSW': 'Golden State Warriors',
#     'HOU': 'Houston Rockets',
#     'IND': 'Indiana Pacers',
#     'LAC': 'Los Angeles Clippers',
#     'LAL': 'Los Angeles Lakers',
#     'MEM': 'Memphis Grizzlies',
#     'MIA': 'Miami Heat',
#     'MIL': 'Milwaukee Bucks',
#     'MIN': 'Minnesota Timberwolves',
#     'NOP': 'New Orleans Pelicans',
#     'NYK': 'New York Knicks',
#     'OKC': 'Oklahoma City Thunder',
#     'ORL': 'Orlando Magic',
#     'PHI': 'Philadelphia 76ers',
#     'PHO': 'Phoenix Suns',
#     'POR': 'Portland Trail Blazers',
#     'SAC': 'Sacramento Kings',
#     'SAS': 'San Antonio Spurs',
#     'TOR': 'Toronto Raptors',
#     'UTA': 'Utah Jazz',
#     'WAS': 'Washington Wizards'
# }

# # Load and preprocess opponent stats data
# def load_opponent_stats(opponent_file):
#     opp_stats = pd.read_csv(opponent_file)
    
#     # A lower rating means better offense (less opponent rebounds etc.)
#     opp_stats['defensive_allowance_rating'] = opp_stats['team_defensive_allowance'] = 1 / (
#                                             0.5 * opp_stats['opp_trb_per_game'] +
#                                             0.3 * opp_stats['opp_stl_per_game'] +
#                                             0.2 * opp_stats['opp_blk_per_game']
#                                             ) * 10
#     # Create a separate dataframe for defensive allowance
#     team_defensive_allowance = opp_stats[['team', 'defensive_allowance_rating']]
    
#     # Clean column values before merging
#     team_defensive_allowance['team'] = team_defensive_allowance['team'].str.strip()

#     return team_defensive_allowance


# # PPG_file = player_per_game
# def load_and_preprocess_player_data(ppg_file, team_defensive_allowance):
#     ppg_data = pd.read_csv(ppg_file)
    
#     # Replace the 'tm' abbreviations with full team names
#     ppg_data['tm'] = ppg_data['tm'].str.strip().map(team_abbreviation_to_name)
    
#     # Check if any 'tm' value didn't match a team
# #     if ppg_data['tm'].isnull().any():
# # #         print("Warning: Some team abbreviations did not match a full team name.")
# # #         print(ppg_data[ppg_data['tm'].isnull()][['player', 'tm']])
    
#     # Merge the player data with the team defensive allowance, using 'tm' from ppg_data and 'team' from team_defensive_allowance
#     ppg_data = pd.merge(ppg_data, team_defensive_allowance, how='left', left_on='tm', right_on='team')


#     # Handle missing values (basic example: fill with mean of each column)
#     ppg_data.fillna(ppg_data.mean(numeric_only=True), inplace=True)

#     # Filter for the most recent season
#     if 'season' in ppg_data.columns:
#         most_recent_season = ppg_data['season'].max()
#         ppg_data = ppg_data[ppg_data['season'] == most_recent_season]
#     else:
#         raise ValueError("The dataset does not contain a 'season' column.")
    
#     features = [
#         'age', 'experience', 
#         'x2p_per_game', 'ft_per_game', 'trb_per_game', 'ast_per_game', 'defensive_allowance_rating',
#         'mp_per_game', 'fg_per_game', 'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game', 
#         'x3p_percent', 'x2p_per_game', 'x2pa_per_game', 'x2p_percent', 'e_fg_percent', 'ft_per_game',
#         'fta_per_game', 'ft_percent', 'orb_per_game', 'drb_per_game', 'stl_per_game', 
#         'blk_per_game', 'tov_per_game', 'pf_per_game'
#     ]

#     return ppg_data, features

# # Normalize features
# def normalize_features(X):
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
#     return X_scaled, scaler

# # Predict player performance
# def predict_player_performance(data, model, scaler, player_name, target):
#     ppg_data = data[data['player'] == player_name]
#     if ppg_data.empty:
#         return f"Player {player_name} not found in the dataset."

#     features = [
#         'age', 'experience', 
#         'x2p_per_game', 'ft_per_game', 'trb_per_game', 'ast_per_game', 'defensive_allowance_rating',
#         'mp_per_game', 'fg_per_game', 'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game', 
#         'x3p_percent', 'x2p_per_game', 'x2pa_per_game', 'x2p_percent', 'e_fg_percent', 'ft_per_game',
#         'fta_per_game', 'ft_percent', 'orb_per_game', 'drb_per_game', 'stl_per_game', 
#         'blk_per_game', 'tov_per_game', 'pf_per_game'
#     ]

#     player_features = ppg_data[features].values
#     player_features_scaled = scaler.transform(player_features)
#     predicted_value = model.predict(player_features_scaled)

#     return f"Predicted {target} for {player_name}: {predicted_value[0]:.2f}"

# # Train the model for a specific target
# def train_model(data, features, target):
#     X = data[features]
#     y = data[target]

#     # Handle missing values in X (if any)
#     X.fillna(X.mean(), inplace=True)  # Fill missing values with column means

#     # Normalize features
#     X_scaled, scaler = normalize_features(X)

#     # Train the model
#     model = LinearRegression()
#     model.fit(X_scaled, y)

#     return model, scaler

# # Main workflow
# def main():
#     # File paths
#     opponent_file = '/Users/davislaroque/Desktop/NBA Project/archive/Opponent Stats Per Game.csv'
#     ppg_file = '/Users/davislaroque/Desktop/NBA Project/archive/Player Per Game.csv'

#     # Load and preprocess data
#     team_defensive_allowance = load_opponent_stats(opponent_file)
#     player_data, features = load_and_preprocess_player_data(ppg_file, team_defensive_allowance)

#     # CHANGE TARGET VARIABLE
#     target = 'pts_per_game'
#     # CHANGE PLAYER NAME
#     player_name = "Austin Reaves"

#     # Train the model for the specified target
#     model, scaler = train_model(player_data, features, target)

#     # Predict player performance
#     result = predict_player_performance(player_data, model, scaler, player_name, target)
#     print(result)

# if __name__ == '__main__':
#     main()


In [14]:
# #Version #3

# from scipy.stats import norm
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression

# team_abbreviation_to_name = {
#     'ATL': 'Atlanta Hawks',
#     'BOS': 'Boston Celtics',
#     'BRK': 'Brooklyn Nets',
#     'CHO': 'Charlotte Hornets',
#     'CHI': 'Chicago Bulls',
#     'CLE': 'Cleveland Cavaliers',
#     'DAL': 'Dallas Mavericks',
#     'DEN': 'Denver Nuggets',
#     'DET': 'Detroit Pistons',
#     'GSW': 'Golden State Warriors',
#     'HOU': 'Houston Rockets',
#     'IND': 'Indiana Pacers',
#     'LAC': 'Los Angeles Clippers',
#     'LAL': 'Los Angeles Lakers',
#     'MEM': 'Memphis Grizzlies',
#     'MIA': 'Miami Heat',
#     'MIL': 'Milwaukee Bucks',
#     'MIN': 'Minnesota Timberwolves',
#     'NOP': 'New Orleans Pelicans',
#     'NYK': 'New York Knicks',
#     'OKC': 'Oklahoma City Thunder',
#     'ORL': 'Orlando Magic',
#     'PHI': 'Philadelphia 76ers',
#     'PHO': 'Phoenix Suns',
#     'POR': 'Portland Trail Blazers',
#     'SAC': 'Sacramento Kings',
#     'SAS': 'San Antonio Spurs',
#     'TOR': 'Toronto Raptors',
#     'UTA': 'Utah Jazz',
#     'WAS': 'Washington Wizards'
# }



# #Function that loads team file 
# def load_and_preprocess_player_data(ppg_file, team_defensive_allowance):
#     ppg_data = pd.read_csv(ppg_file)
    
#     # Replace the 'tm' abbreviations with full team names
#     ppg_data['tm'] = ppg_data['tm'].str.strip().map(team_abbreviation_to_name)
    
    
#     # Merge the player data with the team defensive allowance, using 'tm' from ppg_data and 'team' from team_defensive_allowance
#     ppg_data = pd.merge(ppg_data, team_defensive_allowance, how='left', left_on='tm', right_on='team')


#     # Handle missing values (basic example: fill with mean of each column)
#     ppg_data.fillna(ppg_data.mean(numeric_only=True), inplace=True)

#     # Filter for the most recent season
#     if 'season' in ppg_data.columns:
#         most_recent_season = ppg_data['season'].max()
#         ppg_data = ppg_data[ppg_data['season'] == most_recent_season]
#     else:
#         raise ValueError("The dataset does not contain a 'season' column.")
    
#     features = [
#     'age', 'experience', 
#     'x2p_per_game', 'ft_per_game', 'trb_per_game', 'ast_per_game', 'defensive_allowance_rating',
#     'mp_per_game', 'fg_per_game', 'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game', 
#     'x3p_percent', 'x2pa_per_game', 'x2p_percent', 'e_fg_percent', 
#     'fta_per_game', 'ft_percent', 'orb_per_game', 'drb_per_game', 'stl_per_game', 
#     'blk_per_game', 'tov_per_game', 'pf_per_game'
# ]

#     return ppg_data, features


# def predict_with_prediction_interval(model, scaler, X_train_scaled, sample_features, alpha=0.05):
#     from scipy.stats import norm

#     # Scale the sample features (must be 2D)
#     sample_features_scaled = scaler.transform(sample_features)
#     predicted_value = model.predict(sample_features_scaled)[0]

#     # z-score for the desired confidence level
#     z_score = norm.ppf(1 - alpha / 2)

#     # Calculate (X^T X)^-1
#     XTX_inv = np.linalg.inv(X_train_scaled.T @ X_train_scaled)

#     # Calculate the variance for the new prediction
#     x_0 = sample_features_scaled[0].reshape(-1, 1)  # column vector
#     variance = model.sigma_**2 + (x_0.T @ XTX_inv @ x_0)[0, 0]

#     # Margin of error
#     margin_error = z_score * np.sqrt(variance)

#     lower_bound = predicted_value - margin_error
#     upper_bound = predicted_value + margin_error

#     return predicted_value, lower_bound, upper_bound


# #Function that loads opponents stats
# def load_opponent_stats(opponent_file):
#     opp_stats = pd.read_csv(opponent_file)
    
#     # A lower rating means better offense (less opponent rebounds etc.)
#     opp_stats['defensive_allowance_rating'] = opp_stats['team_defensive_allowance'] = 1 / (
#                                             0.5 * opp_stats['opp_trb_per_game'] +
#                                             0.3 * opp_stats['opp_stl_per_game'] +
#                                             0.2 * opp_stats['opp_blk_per_game']
#                                             ) * 10
#     # Create a separate dataframe for defensive allowance
#     team_defensive_allowance = opp_stats[['team', 'defensive_allowance_rating']]
    
#     # Clean column values before merging
#     team_defensive_allowance = team_defensive_allowance.copy()
#     team_defensive_allowance['team'] = team_defensive_allowance['team'].str.strip()

#     return team_defensive_allowance



# def main():
#     # File paths
#     opponent_file = '/Users/davislaroque/Desktop/NBA Project/archive/Opponent Stats Per Game.csv'
#     ppg_file = '/Users/davislaroque/Desktop/NBA Project/archive-4-15-25/Player Per Game.csv'
    
#     # CHANGE TARGET VARIABLE
#     target = 'trb_per_game'
#     # CHANGE PLAYER NAME
#     player_name = "Bam Adebayo"
    
#     alpha = 0.05  # 95% prediction interval

#     # Load and preprocess data
#     team_defensive_allowance = load_opponent_stats(opponent_file)
#     player_data, features = load_and_preprocess_player_data(ppg_file, team_defensive_allowance)

#     # Train model
#     model, scaler = train_model_with_error_estimation(player_data, features, target)

#     # Scale full training set (needed for prediction interval)
#     X_train_scaled, _ = normalize_features(player_data[features])

#     # Grab the player row
#     player_row = player_data[player_data['player'] == player_name]
#     if player_row.empty:
#         print(f"Player {player_name} not found in the dataset.")
#         return

#     # Grab player features
#     sample_features = player_row[features].values[:1]

#     # Get prediction interval
#     predicted_value, lower_bound, upper_bound = predict_with_prediction_interval(
#         model, scaler, X_train_scaled, sample_features, alpha=alpha
#     )

#     # Print result
#     print(f"Predicted {target} for {player_name}: {predicted_value:.2f}")
#     print(f"If {player_name} plays a game under similar conditions, we expect him to get between {lower_bound:.2f} and {upper_bound:.2f} {target}, with 95% confidence.")


# if __name__ == "__main__":
#     main()

In [1]:
# #Version #4 with random forest and linear regression

# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import StandardScaler
# import pandas as pd
# import numpy as np
# from scipy.stats import norm

# # Normalization helper
# def normalize_features(X):
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
#     return X_scaled, scaler

# # Train/Test Split and Model Training with Random Forest and Linear Regression
# def train_models_with_comparison(data, features, target):
#     X = data[features]
#     y = data[target]

#     # Fill and scale features
#     X = X.fillna(X.mean())
#     X_scaled, scaler = normalize_features(X)

#     # Split data
#     X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#     # Linear Regression
#     lr_model = LinearRegression()
#     lr_model.fit(X_train, y_train)
#     lr_train_preds = lr_model.predict(X_train)
#     lr_residuals = y_train - lr_train_preds
#     lr_model.sigma_ = np.std(lr_residuals)
    
#     # Random Forest
#     rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
#     rf_model.fit(X_train, y_train)

#     # Evaluate
#     lr_mse = mean_squared_error(y_test, lr_model.predict(X_test))
#     rf_mse = mean_squared_error(y_test, rf_model.predict(X_test))

#     print(f"Linear Regression MSE: {lr_mse:.4f}")
#     print(f"Random Forest MSE: {rf_mse:.4f}")

#     return lr_model, rf_model, scaler, X_train, X_test



# # Prediction Interval for Linear Regression
# def predict_with_prediction_interval(model, scaler, X_train_scaled, sample_features, alpha=0.05):
#     sample_features_scaled = scaler.transform(sample_features)
#     predicted_value = model.predict(sample_features_scaled)[0]
#     z_score = norm.ppf(1 - alpha / 2)

#     XTX_inv = np.linalg.inv(X_train_scaled.T @ X_train_scaled)
#     x_0 = sample_features_scaled[0].reshape(-1, 1)
#     variance = model.sigma_**2 + (x_0.T @ XTX_inv @ x_0)[0, 0]

#     margin_error = z_score * np.sqrt(variance)
#     lower_bound = predicted_value - margin_error
#     upper_bound = predicted_value + margin_error

#     return predicted_value, lower_bound, upper_bound


# #Function that loads opponents stats
# def load_opponent_stats(opponent_file):
#     opp_stats = pd.read_csv(opponent_file)
    
#     # A lower rating means better offense (less opponent rebounds etc.)
#     opp_stats['defensive_allowance_rating'] = opp_stats['team_defensive_allowance'] = 1 / (
#                                             0.5 * opp_stats['opp_trb_per_game'] +
#                                             0.3 * opp_stats['opp_stl_per_game'] +
#                                             0.2 * opp_stats['opp_blk_per_game']
#                                             ) * 10
#     # Create a separate dataframe for defensive allowance
#     team_defensive_allowance = opp_stats[['team', 'defensive_allowance_rating']]
    
#     # Clean column values before merging
#     team_defensive_allowance = team_defensive_allowance.copy()
#     team_defensive_allowance['team'] = team_defensive_allowance['team'].str.strip()

#     return team_defensive_allowance

# # Simple prediction from Random Forest
# def predict_rf(rf_model, scaler, sample_features):
#     sample_scaled = scaler.transform(sample_features)
#     return rf_model.predict(sample_scaled)[0]

# def load_and_preprocess_player_data(ppg_file, team_defensive_allowance):
#     ppg_data = pd.read_csv(ppg_file)
    
#     # Replace the 'tm' abbreviations with full team names
#     ppg_data['tm'] = ppg_data['tm'].str.strip().map(team_abbreviation_to_name)
    
#     # Interaction terms based on mintues played
#     ppg_data['mp_pts'] = ppg_data['mp_per_game'] * ppg_data['pts_per_game']
#     ppg_data['mp_trb'] = ppg_data['mp_per_game'] * ppg_data['trb_per_game']
    
#     # Merge the player data with the team defensive allowance, using 'tm' from ppg_data and 'team' from team_defensive_allowance
#     ppg_data = pd.merge(ppg_data, team_defensive_allowance, how='left', left_on='tm', right_on='team')


#     # Handle missing values (basic example: fill with mean of each column)
#     ppg_data.fillna(ppg_data.mean(numeric_only=True), inplace=True)

#     # Filter for the most recent season
#     if 'season' in ppg_data.columns:
#         most_recent_season = ppg_data['season'].max()
#         ppg_data = ppg_data[ppg_data['season'] == most_recent_season]
#     else:
#         raise ValueError("The dataset does not contain a 'season' column.")
    
#     features = [
#     'age', 'experience', 
#     'x2p_per_game', 'ft_per_game', 'trb_per_game', 'ast_per_game', 'defensive_allowance_rating',
#     'mp_per_game', 'fg_per_game', 'fga_per_game', 'fg_percent', 'x3p_per_game', 'x3pa_per_game', 
#     'x3p_percent', 'x2pa_per_game', 'x2p_percent', 'e_fg_percent', 
#     'fta_per_game', 'ft_percent', 'orb_per_game', 'drb_per_game', 'stl_per_game', 
#     'blk_per_game', 'tov_per_game', 'pf_per_game', 'mp_pts', 'mp_trb'
# ]

#     return ppg_data, features


# def calculate_expected_value_from_prediction(
#     predicted_points, sigma, sportsbook_line, odds, over=True
# ):
#     # Calculate probability from normal distribution
#     if over:
#         prob_win = 1 - norm.cdf(sportsbook_line, loc=predicted_points, scale=sigma)
#     else:
#         prob_win = norm.cdf(sportsbook_line, loc=predicted_points, scale=sigma)
    
#     prob_loss = 1 - prob_win

#     # Convert odds to decimal
#     if odds > 0:
#         payout_ratio = odds / 100
#     else:
#         payout_ratio = 100 / abs(odds)

#     # Expected value, assuming $100 bet
#     ev = (prob_win * payout_ratio * 100) - (prob_loss * 100)
    
#     return ev, prob_win





# def main():
#     # File paths
#     opponent_file = '/Users/davislaroque/Desktop/NBA Project/archive/Opponent Stats Per Game.csv'
#     ppg_file = '/Users/davislaroque/Desktop/NBA Project/archive-4-15-25/Player Per Game.csv'
    
#     # CHANGE TARGET VARIABLE
#     target = 'pts_per_game'
#     # CHANGE PLAYER NAME
#     player_name = "Pascal Siak"
    
#     sportsbook_line = 16.5   # CHANGE TO ACTUAL LINE
#     odds = -115              # CHANGE TO ACTUAL ODDS
    
    
#     alpha = 0.05  # 95% prediction interval
    
#     # Load and preprocess data
#     team_defensive_allowance = load_opponent_stats(opponent_file)
#     player_data, features = load_and_preprocess_player_data(ppg_file, team_defensive_allowance)

#     lr_model, rf_model, scaler, X_train_scaled, X_test_scaled = train_models_with_comparison(
#     player_data, features, target
#     )
    
#     # Grab the player row
#     player_row = player_data[player_data['player'] == player_name]
#     if player_row.empty:
#         print(f"Player {player_name} not found in the dataset.")
#         return
    
    
#     # get the average minutes and FGA
#     mp_per_game = player_row["mp_per_game"].values[0]
#     fga_per_game = player_row["fga_per_game"].values[0]



#     # Grab player features
#     sample_features = player_row[features].values[:1]

#     # Predict with Linear Regression (with Prediction Interval)
#     lr_pred, lower, upper = predict_with_prediction_interval(lr_model, scaler, X_train_scaled, sample_features)

#     # Predict with Random Forest
#     rf_pred = predict_rf(rf_model, scaler, sample_features)
    
    

#     # We'll re-use the LR prediction and sigma
#     predicted_points = lr_pred
#     sigma = lr_model.sigma_

#     ev, prob_win = calculate_expected_value_from_prediction(
#         predicted_points, sigma, sportsbook_line, odds, over=True
#     )

    
    
#     # Print results
#     print(f"\n--- {player_name} ---")
#     print(f"Averaging {mp_per_game:.1f} minutes with {fga_per_game:.1f} FGA/game\n")
#     print(f"\nEV on Over {sportsbook_line} points at {odds} odds: ${ev:.2f}")
#     print(f"Model-estimated chance of hitting Over: {prob_win * 100:.2f}%\n")
#     print(f"Random Forest Prediction: {rf_pred:.2f} {target}\n")
#     print(f"Linear Regression Prediction: {lr_pred:.2f} {target}")
#     print(f"95% Prediction Interval for LR: ({lower:.2f}, {upper:.2f})")

# if __name__ == "__main__":
#     main()

In [28]:
# import pandas as pd
# stats_by_game = pd.read_csv('/Users/davislaroque/Desktop/NBA Project/game-by-game-stats/PlayerStatistics2025.csv')

# df = pd.DataFrame(stats_by_game)


# # Convert the 'date' column to datetime
# df['gameDate'] = pd.to_datetime(df['gameDate'])

# # Define the date range
# start_date = pd.to_datetime('2024-10-22')
# end_date = pd.to_datetime('2025-06-22')

# # Filter the DataFrame
# filtered_df = df[(df['gameDate'] >= start_date) & (stats_by_game['gameDate'] <= end_date)]




In [29]:
# import pandas as pd

# # Load original
# team_stats_by_game = pd.read_csv('/Users/davislaroque/Desktop/NBA Project/game-by-game-stats/TeamStatistics.csv')
# df = pd.DataFrame(team_stats_by_game)

# # Convert 'gameDate' to datetime
# df['gameDate'] = pd.to_datetime(df['gameDate'])

# # Define the season date range
# start_date = pd.to_datetime('2024-10-22')
# end_date = pd.to_datetime('2025-06-22')

# # Filter to just the 2024-25 season
# filtered_df = df[(df['gameDate'] >= start_date) & (df['gameDate'] <= end_date)]

# # Save to new file
# filtered_df.to_csv('/Users/davislaroque/Desktop/NBA Project/game-by-game-stats/filtered_team_stats.csv', index=False)


In [5]:
##### Version #5 with using individual game data not averages
#Link to kaggle page : https://www.kaggle.com/datasets/eoinamoore/historical-nba-data-and-player-box-scores?select=PlayerStatistics.csv


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from scipy.stats import norm
import math


# Normalization helper
def normalize_features(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, scaler

# Train/Test Split and Model Training with Random Forest and Linear Regression
def train_models_with_comparison(data, features, target):
    X = data[features]
    y = data[target]

    # Fill and scale features
    X = X.fillna(X.mean())
    X_scaled, scaler = normalize_features(X)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Linear Regression
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_train_preds = lr_model.predict(X_train)
    lr_residuals = y_train - lr_train_preds
    lr_model.sigma_ = np.std(lr_residuals)
    
    # Random Forest
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    # Evaluate
    lr_mse = mean_squared_error(y_test, lr_model.predict(X_test))
    rf_mse = mean_squared_error(y_test, rf_model.predict(X_test))

    
    print(f"Random Forest RMSE: {math.sqrt(rf_mse):.4f}")

    return lr_model, rf_model, scaler, X_train, X_test



# Prediction Interval for Linear Regression
def predict_with_prediction_interval(model, scaler, X_train_scaled, sample_features, alpha=0.05):
    sample_features_scaled = scaler.transform(sample_features)
    predicted_value = model.predict(sample_features_scaled)[0]
    z_score = norm.ppf(1 - alpha / 2)

    XTX_inv = np.linalg.pinv(X_train_scaled.T @ X_train_scaled)
    x_0 = sample_features_scaled[0].reshape(-1, 1)
    variance = model.sigma_**2 + (x_0.T @ XTX_inv @ x_0)[0, 0]

    margin_error = z_score * np.sqrt(variance)
    lower_bound = predicted_value - margin_error
    upper_bound = predicted_value + margin_error

    return predicted_value, lower_bound, upper_bound



# Simple prediction from Random Forest
def predict_rf(rf_model, scaler, sample_features):
    sample_scaled = scaler.transform(sample_features)
    return rf_model.predict(sample_scaled)[0]



def load_and_preprocess_player_data(ppg_file, player_name=None):

    ppg_data = pd.read_csv(ppg_file)
    #print("Columns in loaded DataFrame:", ppg_data.columns)
    

    ppg_data['first_last'] = ppg_data['firstName'] + ' ' + ppg_data['lastName']

    if player_name:
        ppg_data = ppg_data[ppg_data['first_last'] == player_name]
        

    ppg_data['gameDate'] = pd.to_datetime(ppg_data['gameDate'])
    ppg_data = ppg_data.sort_values(by=['first_last', 'gameDate'])

    ppg_data['game_month'] = ppg_data['gameDate'].dt.month
    ppg_data['game_dayofweek'] = ppg_data['gameDate'].dt.dayofweek
    ppg_data['days_since_last_game'] = (
        ppg_data.groupby('first_last')['gameDate'].diff().dt.days.fillna(7)
    )

    ppg_data['rolling_avg_points_3'] = (
        ppg_data.groupby('first_last')['points']
        .rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
    )
    ppg_data['rolling_avg_points_5'] = (
        ppg_data.groupby('first_last')['points']
        .rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)
    )

    ppg_data['mp_pts'] = ppg_data['numMinutes'] * ppg_data['points']
    ppg_data['mp_trb'] = ppg_data['numMinutes'] * ppg_data['reboundsTotal']

    ppg_data.fillna(ppg_data.mean(numeric_only=True), inplace=True)

    # Downcast numerics
    for col in ppg_data.select_dtypes(include='number').columns:
        ppg_data[col] = pd.to_numeric(ppg_data[col], downcast='float')

    # Define the features for the model
    features = [
        'numMinutes', 'blocks', 'steals', 'foulsPersonal', 'turnovers', 'days_since_last_game',
        'assists', 'fieldGoalsAttempted', 'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
        'freeThrowsAttempted', 'freeThrowsMade', 'rolling_avg_points_3', 'rolling_avg_points_5',
        'plusMinusPoints', 'reboundsDefensive', 'reboundsOffensive', 'reboundsTotal',
        'mp_pts', 'mp_trb',
        'game_month', 'game_dayofweek', 'playerteamName', 'opponentteamName', 'home'
    ]

    return ppg_data, features






def calculate_expected_value_from_prediction(
    predicted_points, sigma, sportsbook_line, odds, over=True
):
    # Calculate probability from normal distribution
    if over:
        prob_win = 1 - norm.cdf(sportsbook_line, loc=predicted_points, scale=sigma)
    else:
        prob_win = norm.cdf(sportsbook_line, loc=predicted_points, scale=sigma)
    
    prob_loss = 1 - prob_win

    # Convert odds to decimal
    if odds > 0:
        payout_ratio = odds / 100
    else:
        payout_ratio = 100 / abs(odds)

    # Expected value, assuming $100 bet
    ev = (prob_win * payout_ratio * 100) - (prob_loss * 100)
    
    return ev, prob_win



def calculate_betting_edge(blended_mean, std_dev, sportsbook_line, odds, bet_type='over'):
    """
    Calculate expected value and model probability for a given bet.
    """
    # Convert betting odds to implied probability
    if odds > 0:
        vegas_prob = 100 / (odds + 100)
    else:
        vegas_prob = -odds / (-odds + 100)

    # Calculate model probability
    if bet_type.lower() == 'over':
        model_prob = 1 - norm.cdf(sportsbook_line, loc=blended_mean, scale=std_dev)
    else:  # Under
        model_prob = norm.cdf(sportsbook_line, loc=blended_mean, scale=std_dev)

    # Calculate expected value (simple)
    payout_multiplier = odds / 100 if odds > 0 else 100 / abs(odds)
    ev = (model_prob * payout_multiplier) - (1 - model_prob)

    # Calculate model edge
    edge = model_prob - vegas_prob

    return ev, model_prob, vegas_prob, edge




#Functions used to calculate the mean without accounting for fluke / blowout games
def weighted_mean(df, stat_col='points', weight_col='numMinutes'):
    valid = df[(df[weight_col] > 0) & df[stat_col].notna()]
    weighted_sum = (valid[stat_col] * valid[weight_col]).sum()
    total_weight = valid[weight_col].sum()
    return weighted_sum / total_weight if total_weight > 0 else np.nan

def ewma_stat(df, stat_col='points', span=5):
    sorted_df = df.sort_values('gameDate')
    return sorted_df[stat_col].ewm(span=span, adjust=False).mean().iloc[-1]

def blended_player_stat(player_data, stat_col='points', span=5, wm_weight=0.6):
    wm = weighted_mean(player_data, stat_col=stat_col, weight_col='numMinutes')
    ewma = ewma_stat(player_data, stat_col=stat_col, span=span)
    
    if pd.isna(wm) and pd.isna(ewma):
        return np.nan
    elif pd.isna(wm):
        return ewma
    elif pd.isna(ewma):
        return wm
    else:
        return wm_weight * wm + (1 - wm_weight) * ewma
    
    

def adjust_prediction_with_matchup(player_data, opponent_team, player_team, blended_mean, home_game_flag, team_stats, target, print_log):
    """Adjusts the blended mean based on matchup stats, pace, and home/away effects, and prints each adjustment."""

    adjustment_log = []  # collect adjustment notes
    
    
     # --- Step 0: Blended Mean ---
    blended_mean = blended_player_stat(player_data, stat_col=target)
    if pd.isna(blended_mean):
        adjustment_log.append("Initial blended mean: Failed to calculate (NaN)")
        print("Adjustment Breakdown:")
        for adj in adjustment_log:
            print("  •", adj)
        return np.nan
    #adjustment_log.append(f"Initial blended mean: {blended_mean:.2f} {target}")
    
    

    # --- Get player's last 5 games vs opponent ---
    recent_vs_opponent = player_data[player_data['opponentteamName'] == opponent_team].tail(5)
    

    if not recent_vs_opponent.empty:
        opponent_mean = recent_vs_opponent[target].mean()
        overall_mean = player_data[target].mean()
        opponent_adjustment = (opponent_mean - overall_mean) * 0.5  # weight it moderately
        blended_mean += opponent_adjustment
        adjustment_log.append(f"Opponent adjustment: {opponent_adjustment:+.2f} {target}")
    else:
        adjustment_log.append("Opponent adjustment: None (no recent data)")

    # --- Calculate pace adjustment ---
    try:
        # Find team stats for both teams
        team_row = team_stats[team_stats['teamName'] == player_team].iloc[-1]
        
        opponent_row = team_stats[team_stats['opponentTeamName'] == opponent_team].iloc[-1]

        # Calculate total possessions for each team
        team_total_possessions = (
            team_row['fieldGoalsAttempted'] +
            0.44 * team_row['freeThrowsAttempted'] -
            team_row['reboundsOffensive'] +
            team_row['turnovers']
        )
        opponent_total_possessions = (
            opponent_row['fieldGoalsAttempted'] +
            0.44 * opponent_row['freeThrowsAttempted'] -
            opponent_row['reboundsOffensive'] +
            opponent_row['turnovers']
        )

        # Get minutes
        team_total_minutes = team_row['numMinutes']
        opponent_total_minutes = opponent_row['numMinutes']
        
        #different weights for different targets for pace adjustment
        stat_pace_weights = {
            'points': 0.5,
            'reboundsTotal': 0.3,
            'assists': 0.2,
            'steals': 0.1,
            'blocks': 0.1,
            
        }


        

        if pd.notna(team_total_possessions) and pd.notna(opponent_total_possessions) and pd.notna(team_total_minutes) and pd.notna(opponent_total_minutes) and team_total_minutes > 0 and opponent_total_minutes > 0:
            team_pace = (240 / team_total_minutes) * (team_total_possessions)
            
            opponent_pace = (240 / opponent_total_minutes) * (opponent_total_possessions)

            # Average the two paces
            game_pace = (team_pace + opponent_pace) / 2
            #Average game pace
            league_average_pace = 100

            #Gets the difference in average pace with that game pace, gets the corresponding weight from 
            #the feature and adjusts the target prediction 
            pace_diff_percent = (game_pace - league_average_pace) / league_average_pace
            pace_weight = stat_pace_weights.get(target, 0.3)
            pace_adjustment = pace_diff_percent * blended_mean * pace_weight
            blended_mean += pace_adjustment
            
            adjustment_log.append(f"Pace adjustment: {pace_adjustment:+.2f} {target} (Game Pace {game_pace:.1f}, team_pace = {team_pace}, opponent_pace = {opponent_pace}, League Pace {league_average_pace:.1f})")
        else:
            adjustment_log.append(f"Pace adjustment: Skipped because poss = {team_total_possessions}, team_min = {team_total_minutes}, opp_minutes = {opponent_total_minutes}")
    except Exception as e:
        adjustment_log.append(f"Pace adjustment: Failed ({e})")

    # --- Home/Away adjustment ---
    if home_game_flag == 1:
        home_adjustment = 0.02 * blended_mean  # boost 2% for home
        blended_mean += home_adjustment
        adjustment_log.append(f"Home game adjustment: {home_adjustment:+.2f} {target}")
    else:
        away_adjustment = -0.02 * blended_mean  # penalty 2% for away
        blended_mean += away_adjustment
        adjustment_log.append(f"Away game adjustment: {away_adjustment:+.2f} {target}")

    # --- Print all adjustment steps ---
    if print_log:
        print("\nAdjustment Breakdown:")
        for adj in adjustment_log:
            print("  •", adj)

    return blended_mean, adjustment_log


# monte carlo based prediction 
def monte_carlo_adjustment(player_data, opponent_team, player_team, home_game_flag, team_stats, target, sportsbook_line, bet_type='over', n=1000, noise_std=0.05):
    """
    Runs a Monte Carlo simulation by injecting noise into the player's stat distribution and applying the matchup adjustment.
    Returns the empirical probability of hitting the bet line (over/under) and expected value.
    """

    # Precompute the base blended mean
    base_blended_mean = blended_player_stat(player_data, stat_col=target)

    if pd.isna(base_blended_mean):
        print("Cannot run Monte Carlo: blended mean is NaN")
        return None, None

    simulated_predictions = []

    # Generate one noisy sample to print the adjustment log
    noisy_data = player_data.copy()
    noise = np.random.normal(loc=0, scale=noise_std * base_blended_mean, size=len(noisy_data))
    noisy_data[target] = noisy_data[target] + noise
    noisy_data[target] = noisy_data[target].clip(lower=0)

    # Print adjustment log once
    first_adjusted, adjustment_log = adjust_prediction_with_matchup(
        noisy_data, opponent_team, player_team, base_blended_mean, home_game_flag, team_stats, target, print_log=True
    )
    if not pd.isna(first_adjusted):
        simulated_predictions.append(first_adjusted)

    # Run rest of the simulations silently
    for _ in range(n - 1):
        noisy_data = player_data.copy()
        noise = np.random.normal(loc=0, scale=noise_std * base_blended_mean, size=len(noisy_data))
        noisy_data[target] = noisy_data[target] + noise
        noisy_data[target] = noisy_data[target].clip(lower=0)

        adjusted_prediction, _ = adjust_prediction_with_matchup(
            noisy_data, opponent_team, player_team, base_blended_mean, home_game_flag, team_stats, target, print_log=False
        )

        if not pd.isna(adjusted_prediction):
            simulated_predictions.append(adjusted_prediction)

    # Calculate Monte Carlo-based probability
    simulated_predictions = np.array(simulated_predictions)

    if bet_type == 'over':
        prob = np.mean(simulated_predictions > sportsbook_line)
    else:
        prob = np.mean(simulated_predictions < sportsbook_line)

    # Compute expected value (assuming $100 bet)
    payout_ratio = sportsbook_line / 100 if sportsbook_line > 0 else 100 / abs(sportsbook_line)
    ev = (prob * payout_ratio * 100) - ((1 - prob) * 100)

    print(f"Monte Carlo {bet_type.upper()} Prob: {prob:.3f} | Expected Value: ${ev:.2f}")
    return prob, ev



#Imports the excel file and gets the column of residuals to get the standard error of the estimate
def read_xls(residual_file_path, sheet_name=0, **kwargs):
    try:
        df = pd.read_excel(residual_file_path, sheet_name=sheet_name, engine='xlrd', **kwargs)
        return df
    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None







#------------------------------------------------------------------------------------------------------------------------


def main():
    # --- File paths and player/team setup ---
    ppg_file = '/Users/davislaroque/Desktop/NBA Project/game-by-game-stats/filtered_player_data.csv'
    team_stats_file = '/Users/davislaroque/Desktop/NBA Project/game-by-game-stats/filtered_team_stats.csv'
    residual_file_path = '/Users/davislaroque/Downloads/NBA Project Results Summary.xls'

    
    target = 'reboundsTotal'
    player_name = "Anthony Edwards"
    player_team = "Timberwolves"
    opponent_team = "Thunder"
    home_game_flag = 0  # 1 for Home, 0 for Away

    # --- Load and prepare data ---
    player_data, features = load_and_preprocess_player_data(ppg_file, player_name)
    team_stats = pd.read_csv(team_stats_file)
    team_stats = team_stats[pd.to_datetime(team_stats['gameDate']) >= pd.to_datetime("2024-10-22")]
    
    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return
    
    #gets the residuals standard error
    df = read_xls(residual_file_path)
    residuals = df.Residuals
    residuals_std = residuals.std()

    #drop team/categorical columns from model training
    features = [f for f in features if f not in ['playerteamName', 'opponentteamName']]

    
    # Remove target column from features to prevent leakage
    if target == 'reboundsTotal':
        features = [f for f in features if f not in ['reboundsTotal', 'mp_trb']]
    elif target == 'points':
        features = [f for f in features if f not in ['points', 'mp_pts']]
        

    # --- Train models ---
    lr_model, rf_model, scaler, X_train_scaled, X_test_scaled = train_models_with_comparison(
        player_data, features, target
    )

    # --- Prepare latest sample for prediction ---
    player_row = player_data.iloc[[-1]]  # most recent game
    numMinutes = player_row["numMinutes"].values[0]
    fieldGoalsAttempted = player_row["fieldGoalsAttempted"].values[0]
    sample_features = player_row[features].values

    alpha = 0.05  # Confidence level for prediction interval

    # --- Linear Regression prediction with interval ---
    lr_pred, lower, upper = predict_with_prediction_interval(
        lr_model, scaler, X_train_scaled, sample_features
    )

    # --- Random Forest prediction ---
    sample_df = pd.DataFrame(sample_features, columns=features)
    tree_preds = [tree.predict(scaler.transform(sample_df))[0] for tree in rf_model.estimators_]
    rf_pred = np.mean(tree_preds)
    rf_std = np.std(tree_preds)

    # --- Blended Mean (weighted average of LR and RF) ---
    blended_mean = (0.6 * rf_pred) + (0.4 * lr_pred)

    # --- Estimate Standard Deviation (blended) ---
    historical_std = player_data[target].tail(20).std()
    blended_std = (0.6 * historical_std) + (0.4 * rf_std)
    blended_std = max(blended_std, 0.1 * rf_pred)

    # --- Adjust blended_mean with matchup, pace, home/away ---
    adjusted_blended_mean, _ = adjust_prediction_with_matchup(
        player_data=player_data,
        opponent_team=opponent_team,
        player_team=player_team,
        blended_mean=blended_mean,
        home_game_flag=home_game_flag,
        team_stats=team_stats,
        target = target,
        print_log = False
    )

    


    # --- Sportsbook Info (manual input) ---
    sportsbook_line =8.5
    odds = +100
    bet_type = 'over'

    
    
    model_prob, ev = monte_carlo_adjustment(
    player_data=player_data,
    opponent_team=opponent_team,
    player_team=player_team,
    home_game_flag=home_game_flag,
    team_stats=team_stats,
    target=target,
    sportsbook_line=sportsbook_line,
    bet_type=bet_type,
    n=1000,
    noise_std = residuals_std
    )
    
    # --- Calculate EV, model prob, vegas prob, edge ---
    ev, model_prob, vegas_prob, edge = calculate_betting_edge(
        adjusted_blended_mean, blended_std, sportsbook_line, odds, bet_type
    )

    # --- Output ---

    print(f"\n--- {player_name} ---")
    print(f"Averaging {numMinutes:.1f} minutes with {fieldGoalsAttempted:.1f} FGA/game\n")
    print(f"Model predicted mean: {adjusted_blended_mean:.2f} {target} (adjusted)")
    print(f"Model standard deviation: {blended_std:.2f}\n")
    #print(f"Random Forest raw prediction: {rf_pred:.2f} {target}\n")
    #print(f"Linear Regression raw prediction: {lr_pred:.2f} {target}")
    print(f"95% Prediction Interval for Linear Regression: ({lower:.2f}, {upper:.2f})\n")

    print(f"Vegas {bet_type} line: {sportsbook_line} at odds {odds}")
    print(f"Vegas implied probability: {vegas_prob * 100:.2f}%")
    print(f"Model probability of winning bet: {model_prob * 100:.2f}%")
    print(f"Edge over Vegas: {edge * 100:.2f}%")
    print(f"Expected Value (EV): ${ev:.2f}\n")


if __name__ == "__main__":
    main()


/var/folders/43/qtyjg2710fb7jw4r2dclzgqc0000gn/T/ipykernel_29551/4138483687.py:83: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  ppg_data = pd.read_csv(ppg_file)


Random Forest RMSE: 1.3353


/Users/davislaroque/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Adjustment Breakdown:
  • Opponent adjustment: +1.35 reboundsTotal
  • Pace adjustment: +0.05 reboundsTotal (Game Pace 101.0, team_pace = 99.88, opponent_pace = 102.16, League Pace 100.0)
  • Away game adjustment: -0.34 reboundsTotal
Monte Carlo OVER Prob: 0.943 | Expected Value: $2.32

--- Anthony Edwards ---
Averaging 40.0 minutes with 26.0 FGA/game

Model predicted mean: 8.51 reboundsTotal (adjusted)
Model standard deviation: 2.10

95% Prediction Interval for Linear Regression: (7.82, 10.18)

Vegas over line: 8.5 at odds 100
Vegas implied probability: 50.00%
Model probability of winning bet: 50.21%
Edge over Vegas: 0.21%
Expected Value (EV): $0.00



In [ ]:
#RF and LR model summarys


#    import statsmodels.api as sm

#     X = player_data[features].fillna(player_data[features].mean())
#     X = sm.add_constant(X)  # adds intercept
#     y = player_data[target]

#     model = sm.OLS(y, X).fit()
#     print(model.summary())
    
#     import matplotlib.pyplot as plt

#     feature_importance = rf_model.feature_importances_
#     sorted_idx = np.argsort(feature_importance)[::-1]

#     plt.figure(figsize=(10, 6))
#     plt.barh([features[i] for i in sorted_idx], feature_importance[sorted_idx])
#     plt.xlabel("Feature Importance")
#     plt.title("Random Forest Feature Importances")
#     plt.gca().invert_yaxis()
#     plt.show()